Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [34]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [35]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [36]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [37]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts train samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts train samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [38]:
# windowing function
def create_windows_np(data, window_size, horizon):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)
    return X, y

In [39]:
past, future = (5, 2)
X_train, y_train = create_windows_np(data = train_data, window_size = past, horizon = future)
X_test, y_test = create_windows_np(data = test_data, window_size = past, horizon = future)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [40]:
inputs = keras.layers.Input(shape = (past, 1), name = 'Input')

# Capas GRU:
gru_1 = keras.layers.GRU(64, return_sequences = True, name = 'GRU1')(inputs)
gru_2 = keras.layers.GRU(32, name = 'GRU2')(gru_1)

# Outputs:
outputs = keras.layers.Dense(1, name = 'Output')(gru_2)

model = keras.Model(inputs = inputs, outputs = outputs)
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = 'mse',
    metrics = ['mae'])

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU1 (GRU)                      │ (None, 5, 64)          │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU2 (GRU)                      │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,305 (87.13 KB)

 Trainable params: 22,305 (87.13 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 87.9182 - mae: 8.2154 - val_loss: 33.5168 - val_mae: 4.7416
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24.1005 - mae: 3.9356 - val_loss: 24.4244 - val_mae: 3.9572
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 19.8102 - mae: 3.5203 - val_loss: 20.7139 - val_mae: 3.6389
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17.6116 - mae: 3.3354 - val_loss: 18.6696 - val_mae: 3.4542
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 15.5370 - mae: 3.1122 - val_loss: 16.6357 - val_mae: 3.2128
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 13.5547 - mae: 2.8722 - val_loss: 14.3119 - val_mae: 2.9613
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 12.1728 - mae: 2.7117 - val_loss: 12.5872 - val_mae: 2.7594
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11.7400 - mae: 2.6515 - val_loss: 11.3962 - val_mae: 2.6194
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/

In [42]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.9076 - mae: 2.0529
Test Loss: [7.037185192108154, 2.0682549476623535]


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [43]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [44]:
## Create windows

# Multivariate windowing function:
def create_windows_multivariate_np(data, window_size, horizon, target_col_idx):
    if isinstance(data, pd.DataFrame):
        data = data.values
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)
    return X, y


X_train, y_train = create_windows_multivariate_np(data = train_data, window_size = past, horizon = future, target_col_idx = 0)
X_test, y_test = create_windows_multivariate_np(data = test_data, window_size = past, horizon = future, target_col_idx = 0)

In [45]:
# Inputs:
inputs = keras.layers.Input(shape = (past, 2), name = 'Input')

# Capas GRU:
gru_1 = keras.layers.GRU(64, return_sequences = True, name = 'GRU1')(inputs)
gru_2 = keras.layers.GRU(32, name = 'GRU2')(gru_1)

# Outputs:
outputs = keras.layers.Dense(1, name = 'Output')(gru_2)

model = keras.Model(inputs = inputs, outputs = outputs)
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = 'mse',
    metrics = ['mae'])
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU1 (GRU)                      │ (None, 5, 64)          │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GRU2 (GRU)                      │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,497 (87.88 KB)

 Trainable params: 22,497 (87.88 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 92.2403 - mae: 8.5536 - val_loss: 40.2540 - val_mae: 5.2919
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.0686 - mae: 4.2770 - val_loss: 28.2198 - val_mae: 4.2818
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 21.3238 - mae: 3.6721 - val_loss: 22.9944 - val_mae: 3.8329
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 17.3599 - mae: 3.3015 - val_loss: 20.0261 - val_mae: 3.5830
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.3383 - mae: 3.2986 - val_loss: 18.3912 - val_mae: 3.4316
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.4021 - mae: 3.1996 - val_loss: 15.8521 - val_mae: 3.1257
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 13.7165 - mae: 2.8516 - val_loss: 13.9335 - val_mae: 2.9090
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 11.8359 - mae: 2.6878 - val_loss: 13.0271 - val_mae: 2.8057
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms

In [47]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7767 - mae: 1.8653
Test Loss: [6.1524505615234375, 1.9186064004898071]


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


Opciones a, b y d. En todas ellas se emplean varios inputs (palabras, características de la voz y características de las piezas musicales respectivamente) para dar lugar a una única salida (una clasificación de 'sentimiento', una decisión binaria y una clasificación por autor respectivamente).

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.



Todas las respuestas son correctas:

a) Los word embaddings representan las palabras en muchas menos dimensiones que el one-hot encoding.

b) Las palabras similares se representan con vectores similares. En cambio con one-hot encoding se construyen vectores ortogonales.

c) Los embeddings ya entrenados pueden ser usados en otros campos.

d) Verdadero.